[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [3]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve = SECP256k1)
public_key = private_key.get_verifying_key()

message = b"je suis mojo"
signature = private_key.sign(message)

In [78]:
private_key

In [79]:
public_key

In [80]:
message

b'je suis mojo'

In [81]:
public_key.verify(signature, message)

True

In [82]:
public_key.verify(signature, b'je suis wodjo')

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [ ]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class ECDSACoin:
    
    def __init__(self, transfers):
        self.transfers = transfers

In [4]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [ ]:
from utils import serialize

def issue(public_key):
    message = serialize(public_key)
    
    signature = bank_private_key.sign(message)
    transfer = Transfer(
        signature = signature,
        public_key = public_key
    )
    
    coin = ECDSACoin([transfer])
    return coin

# Validating the First Transfer

In [ ]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    

In [ ]:
alice_coin = issue(alice_public_key)

In [ ]:
validate(alice_coin)

In [ ]:
message = serialize(bob_public_key)
signature = bob_private_key.sign(message)
transfer = Transfer(
    signature = signature,
    public_key = bob_public_key
)
alt_coin = ECDSACoin([transfer])

In [ ]:
from ecdsa import BadSignatureError

try:
    validate(alt_coin)
except BadSignatureError:
    print('bad signature')

# Validating Subsequent Transfers

In [ ]:
def transfer_message(previous_signature, next_owner_public_key):
    return serialize({
        "previous_signature":previous_signature,
        "next_owner_public_key":next_owner_public_key
    })

def validate(coin):
    # Check the first transfer 
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    #check the rest of the coin transfer
    previous_transfer = coin.transfers[0]
    for next_transfer in coin.transfers[1:]:
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.public_key.verify(
            next_transfert.signature,
            message
            )

In [ ]:
def get_owner(coin):
    database = {
        serialize(alice_public_key) : "Alice",
        serialize(bob_public_key) : "Bob",
        serialize(bank_public_key) : "Bank",
        }
    lst_pbkey =  coin.transfers[-1].public_key
    return database[serialize(lst_pbkey)]

In [ ]:
coin = issue(alice_public_key)

print("this coin is owned by", get_owner(coin))

message = transfer_message(coin.transfers[-1].signature, bob_public_key)
alice_to_bob = Transfer(
    signature = alice_private_key.sign(message),
    public_key = bob_public_key
)

coin.transfers.append(alice_to_bob)

print("this coin is owned by", get_owner(coin))

message = transfer_message(coin.transfers[-1].signature, bank_public_key)
bob_to_bank = Transfer(
    signature = bob_private_key.sign(message),
    public_key = bank_public_key
)

coin.transfers.append(bob_to_bank)

print("this coin is owned by", get_owner(coin))




# Serialization

In [ ]:
from utils import from_disk, to_disk

In [ ]:
coin = issue(alice_public_key)

In [ ]:
import os

In [ ]:
filename = "alice.ecdsacoin"

In [ ]:
#is the file on disk?
print("Is the coin on disk :",os.path.isfile(filename))


In [ ]:
to_disk(coin, filename)

In [ ]:
#is the file on disk?
print("Is the coin on disk :",os.path.isfile(filename))

In [ ]:
coin = from_disk(filename)
coin

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [14]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = ecdsacoin.Transfer(
    signature=alice_private_key.sign(ecdsacoin.transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()

In [7]:
import importlib

In [8]:
from rewrite import ecdsacoin_hl2
importlib.reload(ecdsacoin_hl2)

<module 'rewrite.ecdsacoin_hl2' from '/Users/hedi/Documents/11-Buidl/digital-cash/rewrite/ecdsacoin_hl2.py'>

In [9]:
dir(ecdsacoin_hl2)

['ECDSACoin_HL',
 'SECP256k1',
 'SigningKey',
 'Transfer_HL',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bank_private_key',
 'bank_public_key',
 'serialize',
 'transfer_message']

In [11]:
coin = ecdsacoin_hl2.ECDSACoin_HL.issue(alice_public_key)
coin.validate()

alice_to_bob = ecdsacoin_hl2.Transfer_HL(
    signature=alice_private_key.sign(ecdsacoin_hl2.transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()

In [12]:
coin